# Collision Avoidance - Live Demo
In this notebook we'll use the model we trained to detect whether the robot is free or blocked to enable a collision avoidance behavior on the robot. It is assumed that the optimized neural network has already been created and loaded into the Jupyter Lab file browser.

This demonstration showcases the implementation of a strategy that enables the JetRacer to reverse when an obstacle is too close and cannot be circumvented by simply steering. This capability was achieved by mounting an ultrasonic distance sensor on the JetRacer. In order to achieve smooth maneuver transitions and maintain a consistent speed during maneuvering, a PID controller was integrated for speed regulation. This controller utilizes an IR speed sensor mounted on the JetRacer. Speed and distance values are computed by an ESP-32 microcontroller connected to the sensors. These measurements are transmitted to the JetRacer via serial communication.

> The employed PID controller is the one implemented in the [Discrete-TimePID](https://github.com/chentyra/Discrete-TimePID) repository.

<img src="https://i.ebayimg.com/images/g/BIYAAOSwBLlVAQnC/s-l1600.jpg" width="300" height="300">
<img src="https://digitalelectronics.lk/wp-content/uploads/2022/02/Infrared-Speed-Sensor-Module4.jpg" width="300" height="300">

## TensorRT

In [ ]:
import cv2
import torch
from torchvision.models import resnet18
from torch2trt import TRTModule
from jetracer.nvidia_racecar import NvidiaRacecar
from jetcam.csi_camera import CSICamera
from utils import preprocess
import torch.nn.functional as F
import numpy as np
import subprocess
import serial 
import time
from discretepid import PID

Load the optimized model by executing the cell below.

In [ ]:
model_trt = TRTModule()
model_trt.load_state_dict(torch.load('best_model_resnet18_trt.pth'))

Inizialize `car` and `camera` objects.

In [ ]:
car = NvidiaRacecar()
camera = CSICamera(width=224, height=224, capture_fps=65)

Enable the serial communication with ESP-32 microcontroller.

In [ ]:
# Set up the serial port
sudo_password = 'jetson'
command = 'chmod 666 /dev/ttyTHS1'
command = command.split()

cmd1 = subprocess.Popen(['echo', sudo_password], stdout=subprocess.PIPE)
cmd2 = subprocess.Popen(['sudo', '-S'] + command, stdin=cmd1.stdout, stdout=subprocess.PIPE)

output = cmd2.stdout.read().decode() 
print(output)

serial_port = serial.Serial(
    baudrate=115200,
    port="/dev/ttyTHS1",
    bytesize=serial.EIGHTBITS,
    parity=serial.PARITY_NONE,
    stopbits=serial.STOPBITS_ONE,
    timeout=0
)

In the scenario described, the speed targets for forward and backward maneuvers differ due to the programming of the ESC regulators, which typically limit reverse speed compared to forward speed. Additionally, when reversing, the ESC is configured to initiate movement only after receiving specific commands.

### Forward to Reverse Maneuver

For a transition from forward to reverse movement, the following steps are required:

- **Stop the Robot**: Issue a command to halt the robot's movement.
- **Introduce a 1-Second Delay**: Implement a 1-second pause to ensure proper execution of the sequence.
- **Send Negative Throttle Value**: Transmit a negative throttle value to the ESC, instructing it to reverse the motor direction.
- **Introduce a 1-Second Delay**: Implement a 1-second pause to ensure proper execution of the sequence.
- **Stop the Robot**: Once again, issue a command to bring the robot to a complete stop.
- **Introduce a 1-Second Delay**: Implement a 1-second pause to ensure proper execution of the sequence.

After all these steps, the robot will commence backward movement.

### Reverse to Forward Maneuver

The transition from reverse to forward movement follows a similar pattern, with the exception of the throttle value:

- **Stop the Robot**: Halt the robot's movement.
- **Introduce a 1-Second Delay**: Implement a 1-second pause to ensure proper execution of the sequence.
- **Send Positive Throttle Value**: Transmit a positive throttle value to the ESC, instructing it to reverse the motor direction and initiate forward movement.
- **Introduce a 1-Second Delay**: Implement a 1-second pause to ensure proper execution of the sequence.
- **Stop the Robot**: Briefly stop the robot's movement.
- **Introduce a 1-Second Delay**: Implement a 1-second pause to ensure proper execution of the sequence.

After all these steps, the robot will resume forward movement.

In [ ]:
# Define PI parameters for forward movement
Kp= 0.25
Ti = 1.0
setpoint = 0.2  # [m/s]
max_value = 0.25
pid = PID(Kp, Ti, setpoint=setpoint, output_limits=(-0.35, max_value))
 
# Define thresholds and flags
distance_threshold = 35  
threshold = 0.5

turning = False  # Flag to indicate if the car is turning
turn_time = time.time()  # Time when the car should stop turning
forward_flag = True  # Flag to distinguish the two situations
prev_forward_flag = forward_flag
backward_timer = time.time()
backward_manoeuvre = False
first_time_backward = True
first_time_turn = True
timer_start = False

try:
    while True:
        image = camera.read()
        x = preprocess(image)
        y = model_trt(x)
        y = F.softmax(y, dim=1)
        prob_blocked = float(y.flatten()[0])
 
        # Read data from the serial port
        serial_port.write(("\n").encode())
        data = serial_port.readline().decode().strip()
 
        if data:
            values = data.split(',')
            distances = [float(value) for value in values if value]
 
            if distances:
                speed = distances[0]  # Get speed
                current_distance = distances[1]  # Get distances
 
                if current_distance <= distance_threshold:  # If obstacle is too close
                    backward_manoeuvre = True
                    turning = False
                    prev_forward_flag = forward_flag
                    forward_flag = False
                    car.steering = 0.0
                    if first_time_backward:
                        backward_timer = time.time()
                        first_time_backward = False
 
                if prob_blocked < threshold and not turning and current_distance >= distance_threshold and not backward_manoeuvre:
                    prev_forward_flag = forward_flag
                    forward_flag = True
                    car.steering = 0.0
                    turning = False
                        
                if prob_blocked >= threshold and not turning and current_distance >= distance_threshold and not backward_manoeuvre:
                    forward_flag = True 
                    car.steering = -0.8  # Steering left
                    turning = True  # Set the turning flag
                    if first_time_turn:
                        turn_time = time.time()  # Start timer
                        first_time_turn = False
 
                if turning and time.time() - turn_time <= 2:
                        car.steering = -0.8  # Steering left
                else:
                    car.steering = 0.0  # Ensure straight movement when not actively turning
                    turning = False
                    first_time_turn = True
 

                if forward_flag != prev_forward_flag:
                    pid.setpoint = 0
                    car.throttle = 0
                    time.sleep(1)
 
                    if forward_flag:
                        car.throttle = 0.15
                    elif not forward_flag:
                        car.throttle = -0.21
                    time.sleep(1)
                     
                    pid.setpoint = 0
                    car.throttle = 0
                    time.sleep(1)
                    prev_forward_flag = forward_flag
 
                if forward_flag and not backward_manoeuvre:
                    pid.setpoint = setpoint
                    car.throttle = pid(speed)
                elif backward_manoeuvre:
                    #print(f"Executing backward manoeuvre block, backward_timer: {backward_timer}")
                    if time.time()- backward_timer < 2:
                        timer_start = True
                        pid.setpoint = -0.3
                        car.throttle = pid(-speed)
                    else:
                        backward_manoeuvre = False
                        first_time_backward = True
                        timer_start = False
except:
    car.throttle = 0
    car.steering = 0